In [23]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from spectral_cube import SpectralCube
from regions import CircleSkyRegion, RectangleSkyRegion
import glob
import os

def crop_fits_to_region(fits_file, l=None, b=None, ra=None, dec=None, width=None, height=None):
    """
    Function to load a FITS file into a spectral cube and crop it to a specified region.
    
    Parameters:
    fits_file (str): Path to the input FITS file.
    l (float, optional): Galactic longitude of the region center in degrees. Default is None.
    b (float, optional): Galactic latitude of the region center in degrees. Default is None.
    ra (float, optional): Right ascension of the region center in degrees. Default is None.
    dec (float, optional): Declination of the region center in degrees. Default is None.
    width (float, optional): Width of the region in degrees. Default is None.
    height (float, optional): Height of the region in degrees. Default is None.

    Returns:
    subcube (SpectralCube): A spectral cube cropped to the specified region.
    """
    # Load the FITS file into a spectral cube
    cube = SpectralCube.read(fits_file)

    if l is not None: 
    
        # Define a circle sky region centered at the specified Galactic longitude and latitude, with a radius equal to the maximum of the specified width and height
        center = SkyCoord(l, b, frame='galactic', unit='deg')
        width = width * u.deg
        height = height * u.deg
        region = RectangleSkyRegion(center, width, height)
        
    elif ra is not None: 
        
        # Define a circle sky region centered at the specified RA and Dec, with a radius equal to the maximum of the specified width and height
        center = SkyCoord(ra, dec, unit='deg')
        width = width * u.deg
        height = height * u.deg
        region = RectangleSkyRegion(center, width, height)
        
    # Crop the spectral cube to the defined region
    subcube = cube.subcube_from_regions([region])

    # Further crop the cube to the minimal enclosing subcube
    subcube = subcube.minimal_subcube()
    
    return subcube

# Define parameters
l = 0.8065474
b = -0.1999737
width = 0.2
height = 0.2

# Directory with the input files
input_dir = '/Users/abarnes/Dropbox/work/Projects/MOPRA/CMZ/3mm/'
output_dir = '../../data/mopra/3mm/'

# Create the output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Find all FITS files in the input directory
fits_files = glob.glob(os.path.join(input_dir, '*.fits'))

# Loop over all FITS files
for fits_file in fits_files:
    # Crop the spectral cube
    subcube = crop_fits_to_region(fits_file, l=l, b=b, width=width, height=height)
    
    # Construct the output filename
    output_file = os.path.join(output_dir, os.path.basename(fits_file))
    
    # Write the cropped cube to a FITS file
    subcube.write(output_file, overwrite=True)